# 继承的优缺点

## 不建议子类化内置类型
- 因为内置类型不会调用定义的类的覆盖的特殊方法，且定义的类继承的方法，也不会调用被覆盖的方法
- 建议继承collections模块，比如UserDict、UserList和UsesrString，这些类做了特殊设计易于扩展

## 多重继承和方法解析顺序
- A是父类，B和C都继承了A，D继承了B,C。那么调用方法的优先顺序如何确定？

In [1]:
class A:
    def ping(self):
        print('ping:', self)


class B(A):
    def pong(self):
        print('pong:', self)


class C(A):
    def pong(self):
        print('PONG:', self)


class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

# 在D上调用pong方法
- D没有实现pong方法，因此得从B和C中选择一个类，运行它的pong方法，测试发现是B
- 也可以指定运行C的pong方法

In [2]:
d = D()

In [3]:
d.pong()

pong: <__main__.D object at 0x000002AE924D1748>


In [4]:
C.pong(d)

PONG: <__main__.D object at 0x000002AE924D1748>


## 使用super
- 使用super()来调用超类方法最安全，如D的ping方法中就先调用了A的ping方法，super()跳过了B和C，因为B和C都没有实现ping方法
- 使用```__mro__```属性可以查看方法解析顺序(Method Resolution Order，MRO），按照方法解析顺序列出各个超类

In [5]:
d.ping()

ping: <__main__.D object at 0x000002AE924D1748>
post-ping: <__main__.D object at 0x000002AE924D1748>


In [7]:
print(D.__mro__)

(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)


## 在D上调用pingpong方法，观察调用
- self.ping()调用的是D的ping方法，该方法先调用super().ping()，即A的ping方法，再输出一行
- super().ping()，同上，直接调用A的ping方法
- self.pong()，调用的是B的pong()方法
- super().pong()，调用的是B的pong()方法，和上面一样，都是按照方法解析顺序查找的
- C.pong(self)，忽略mro，直接运行C的pong()方法
- **mro不仅考虑继承图，也考虑子类声明中列出超类的顺序，例如如果D的声明为class D(C,B)，则在mro中C就在B的前面**

In [8]:
d.pingpong()

ping: <__main__.D object at 0x000002AE924D1748>
post-ping: <__main__.D object at 0x000002AE924D1748>
ping: <__main__.D object at 0x000002AE924D1748>
pong: <__main__.D object at 0x000002AE924D1748>
pong: <__main__.D object at 0x000002AE924D1748>
PONG: <__main__.D object at 0x000002AE924D1748>


## 处理多重继承的建议
- 把接口继承和实现继承区分开
- 使用抽象基类显式表示接口
- 通过混入重用代码
- 在名称中明确指明混入
- 抽象基类可以作为混入，反过来不成立
- 不要子类化多个具体类
- 为用户提供聚合类